In [39]:
%%writefile mapper1.py


import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'


# Your code for reading stop words here
with open(path) as f1:
    t = f1.readlines()
    stop_words_template = list(word.strip() for word in t)
    

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    
    # Your code for mapper here.

    counts = Counter(words)
    
    for word, word_counts in counts.items():
        
        if word.lower() not in stop_words_template:
            print '%s\t%d' % (word.lower(), word_counts)
    
    
    
    

Overwriting mapper1.py


In [40]:
%%writefile reducer1.py

import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Overwriting reducer1.py


In [41]:
%%writefile mapper2.py


import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    sorted_letters = ''.join(sorted(list(key)))         
    print '%s\t%s\t%d' % (sorted_letters, key, count)
    
    
    
    

Overwriting mapper2.py


In [42]:
%%writefile reducer2.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

group_size = 0
words_count = 0
current_key = 0
words_group = []

for line in sys.stdin:
    try:
        key, word, count = unicode(line.strip()).split('\t')
        count = int(count)
    except ValueError as e:
        continue
    #print '%s\t%s\t%d' % (key, word, count)
    if current_key != key:
        if current_key and group_size > 1:
            print '%d\t%d\t%s' % (words_count, group_size, ','.join(words_group))
        current_key = key
        group_size = 0
        words_count = 0
        words_group = []
    group_size += 1
    words_count += count
    words_group.append(word)
    
if current_key and group_size > 1:
    print '%d\t%d\t%s' % (words_count, group_size, ','.join(words_group))

Overwriting reducer2.py


In [38]:
%%bash

OUT_DIR="coursera_mr_task2"$(date +"%s%6N")
NUM_REDUCERS=8

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming Group Words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py,/datasets/stop_words_en.txt \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 

# Stub code for your job
OUT_DIR2="coursera_mr_task2_2_"$(date +"%s%6N")
NUM_REDUCERS=1
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming Group Words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options="-k1,2 -k2,2" \
    -D stream.num.map.output.key.fields=2 \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR2} > /dev/null 
    
hdfs dfs -cat ${OUT_DIR2}/part-00000 | grep -w "english,helsing,hesling,shengli,shingle"


7820	5	english,helsing,hesling,shengli,shingle


19/02/11 22:47:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/11 22:47:59 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/11 22:48:00 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/11 22:48:00 INFO mapreduce.JobSubmitter: number of splits:2
19/02/11 22:48:00 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548618947485_0016
19/02/11 22:48:00 INFO impl.YarnClientImpl: Submitted application application_1548618947485_0016
19/02/11 22:48:00 INFO mapreduce.Job: The url to track the job: http://f4d8b60eadb1:8088/proxy/application_1548618947485_0016/
19/02/11 22:48:00 INFO mapreduce.Job: Running job: job_1548618947485_0016
19/02/11 22:48:06 INFO mapreduce.Job: Job job_1548618947485_0016 running in uber mode : false
19/02/11 22:48:06 INFO mapreduce.Job:  map 0% reduce 0%
19/02/11 22:48:23 INFO mapreduce.Job:  map 11% reduce 0%
19/02/11 22:48:29 INFO mapreduce.Job:  map 17% reduce 0%
19/02/11 22:48:35 INFO 